In [ ]:
import os
import numpy as np
import torch
import random
import parmed
import nglview as ng
import MDAnalysis as md
from MDAnalysis.analysis import dihedrals
import matplotlib.pyplot as plt

import mymd

In [ ]:
seed = 99
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [ ]:
data_path = 'data/ala15/'
csv_path = 'data/ala15/rslt/'
dcd_path = 'data/ala15/rslt/'

old_pdb_file = os.path.join(data_path, 'ala15.pdb')
pdb_file = os.path.join(data_path, 'ala15_new.pdb')
psf_file = os.path.join(data_path, 'ala15.psf')
prmtop_file = os.path.join(data_path, 'ala15.prmtop')
coor_file = None
xsc_file = None

cutoff = None
T = 300.15
dt_fs = 2
dcd_interval = 100
csv_interval = 50
steps = 50000
min_energy_max_iter = 100
box_size = 100  # can not be too small, e.g. 20 do not work, 100 is good
device = 'cpu'
precision = torch.float
use_centered = True

In [ ]:
# mymd.gen_files_from_pdb(old_pdb_file=old_pdb_file,
#                         psf_file=psf_file,
#                         prmtop_file=prmtop_file,
#                         new_pdb_file=pdb_file)

In [ ]:
mol = mymd.get_molecule(prmtop_file=prmtop_file, pdb_file=pdb_file, coor_file=coor_file, xsc_file=xsc_file)

print('---coords---')
print(mol.coords[:2])
print('-----box-----')
print(mol.box)
print('---charge---')
print(mol.charge[:3])
print('---masses---')
print(mol.masses[:3])
print('----bonds----')
print(mol.bonds[:4])
print('---bonds.shape---')
print(mol.bonds.shape)

In [ ]:
try:
    ff = mymd.PrmtopMolForceField(mol, prmtop_file, allow_unequal_duplicates=False)
except:
    print('False causes error, use True.')
    ff = mymd.PrmtopMolForceField(mol, prmtop_file, allow_unequal_duplicates=True)

print(ff.mol.atomtype[:10])
print(ff.mol.charge[:10])  # same atomtype can have different charge ('C1')
print(set(ff.mol.atomtype))
print(set(ff.params.atom_types))

In [ ]:
system = mymd.System(mol, ff, cutoff=cutoff)
system.set_periodic_box_manual(np.array([box_size, box_size, box_size]).reshape(3, 1))

print(system.bond_params[:5])
print(system.improper_params[0]['params'][:5])
print(system.uni_atom_types == np.unique(mol.atomtype))
print(system.uni_atom_types[system.mapped_atom_types[:6]] == mol.atomtype[:6])
print(system.box, mol.box)
print(system.cutoff)
print(system.external)
print(np.array(system.get_exclusions(types=['bonds']))[:3])
print(np.all(np.vstack((np.array(system.bonds), np.array(system.angles[:, [0, 2]]))) == np.array(system.get_exclusions(types=['bonds', 'angles']))))

In [ ]:
integrator = mymd.VelocityVerletIntegrator(dt_fs)
print(integrator.dt, integrator.dt_fs)

In [ ]:
simulation = mymd.Simulation(mol, system, integrator, device, precision, use_centered=use_centered)
simulation.set_positions(mol.coords)
simulation.set_velocities_to_temperature(T=T)
simulation.update_potentials_and_forces()

csv_reporter = mymd.CSVReporter(csv_path, csv_interval)
simulation.add_reporter(csv_reporter)

print(torch.abs(simulation.vel).mean())
print(simulation.forces.shape)
print(simulation.forces[:5])
print(simulation.forces[-3:])
print(simulation.potentials)
print(simulation.potentials_sum)

In [ ]:
print(simulation.pos[:5])
simulation.minimize_energy(min_energy_max_iter)
print(simulation.pos[:5])

print(simulation.forces[:5])
print(simulation.potentials)
print(simulation.potentials_sum)

In [ ]:
simulation.step(steps, dcd_path, dcd_interval)

In [ ]:
u = md.Universe(psf_file, os.path.join(dcd_path, 'traj.dcd'))
# u = md.Universe(psf_file, os.path.join('data/ala15/rslt_in_report/', 'traj.dcd'))
ng.show_mdanalysis(u, gui=True)

In [ ]:
N_terminus = u.select_atoms('resid 1 and name N')
C_terminus = u.select_atoms('resid 5 and name C')

# go through the whole trajectory and compute distance between them for every frame
dist = []
for frame in u.trajectory:
    dist.append(np.linalg.norm(N_terminus.positions - C_terminus.positions))

# the result is in the dist array    
dist = np.array(dist)
plt.figure(figsize=(15,5))

plt.plot(dist, '-k' )
plt.xlabel('timesteps')
plt.ylabel('end-to-end distance, A')

plt.show()

In [ ]:
ram1 = dihedrals.Ramachandran(u).run(0, 100) 
ram_mid = dihedrals.Ramachandran(u).run(200, 300)
ram2 = dihedrals.Ramachandran(u).run(400, 500)  # maybe not include the last snapshot

# ramachandran plot
fig, ax = plt.subplots(figsize=(8,8))
ram1.plot(ax=ax, color='r', marker='.', label='first 100 snapshots')
ram_mid.plot(ax=ax, color='g', marker='.', label='middle 100 snapshots')
ram2.plot(ax=ax, color='b', marker='.', label='last 100 snapshots')
ax.arrow(20, 20, -40, -40, width=2, head_width=8, head_length=12, fc='b', ec='b')
ax.text(30, 20, 'alpha region', color='blue', fontsize=20)
ax.arrow(20, 150, -40, 0, width=2, head_width=8, head_length=12, fc='k', ec='k')
ax.text(30, 150, 'beta region', fontsize=20)
plt.legend()
plt.show()